In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from sstcam_simulation.utils.efficiency import CameraEfficiency, NSB_FLUX_UNIT
from sstcam_simulation.utils.sipm import SiPMOvervoltage
from sstcam_simulation.data import get_data
from astropy import units as u
from ipywidgets import interactive
from IPython.display import display

# Cherenkov & NSB Spectrum Efficiency

In [2]:
efficiency = CameraEfficiency.from_sstcam()
cherenkov_integral = efficiency._integrate_cherenkov(
    efficiency._cherenkov_diff_flux_on_ground, 
    u.Quantity(300, u.nm), 
    u.Quantity(600, u.nm)
)
efficiency.cherenkov_scale = 100 / cherenkov_integral
scale_pde_wavelength = u.Quantity(410, u.nm)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(efficiency.wavelength, efficiency._cherenkov_diff_flux_on_ground, color='blue', alpha=0.1)
l_cherenkov = ax1.plot(efficiency.wavelength, efficiency._cherenkov_diff_flux_inside_pixel, color='blue', label="Cherenkov")
ax2.plot(efficiency.wavelength, efficiency._nsb_diff_flux_on_ground, color='red', alpha=0.1)
l_nsb = ax2.plot(efficiency.wavelength, efficiency._nsb_diff_flux_inside_pixel, color='red', label="NSB")

nsb_nominal = efficiency.nominal_nsb_rate.to("MHz")
nsb_high = efficiency.maximum_nsb_rate.to("MHz")
cherenkov_pde = efficiency.camera_cherenkov_pde
t_nsb_nominal = ax1.text(0.01, 0.90, f"Nominal NSB Rate = {nsb_nominal:.2f}", transform=ax1.transAxes)
t_nsb_high = ax1.text(0.01, 0.85, f"High NSB Rate = {nsb_high:.2f}", transform=ax1.transAxes)
t_cherenkov_pde = ax1.text(0.01, 0.80, f"Effective Cherenkov PDE = {cherenkov_pde:.2f}", transform=ax1.transAxes)

ax1.set_ylim(0, 0.45)
ax2.set_ylim(0, 35)
ax1.set_xlabel("Wavelength [nm]")
ax1.set_ylabel("Cherenkov photons [100 * 1 / nm]")
ax2.set_ylabel("NSB photons [ 1 / (nm m2 ns sr) ]")
fig.legend()

def slide_pde(pde):
    efficiency.scale_pde(scale_pde_wavelength, pde)
    l_nsb[0].set_ydata(efficiency._nsb_diff_flux_inside_pixel)
    l_cherenkov[0].set_ydata(efficiency._cherenkov_diff_flux_inside_pixel)
    
    nsb_nominal = efficiency.nominal_nsb_rate.to("MHz")
    nsb_high = efficiency.maximum_nsb_rate.to("MHz")
    cherenkov_pde = efficiency.camera_cherenkov_pde
    t_nsb_nominal.set_text(f"Nominal NSB Rate = {nsb_nominal:.2f}")
    t_nsb_high.set_text(f"High NSB Rate = {nsb_high:.2f}")
    t_cherenkov_pde.set_text(f"Effective Cherenkov PDE = {cherenkov_pde:.2f}")

slider = widgets.FloatSlider(
    value=0.5,
    min=0.01,
    max=1,
    step=0.01,
    description=f'PDE @ {scale_pde_wavelength}:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
interactive(slide_pde, pde=slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, description='PDE @ 410.0 nm:', layout=Layout(width='50%'), max=1.…

# Load Requirement Curves

In [3]:
requirement_pde = 0.25
requirement_ph = np.geomspace(4, 4000)
requirement_pe = requirement_ph * requirement_pde

def load(path, interp_x):
    path = get_data(f"datasheet/{path}")
    x, y = np.loadtxt(path, unpack=True)
    return np.interp(interp_x, x, y)

requirement_nominal_nsb = load("IntensityRes.txt", requirement_ph)
requirement_high_nsb = load("IntensityResHighNSB.txt", requirement_ph)
requirement_r1_nominal_nsb = load("IntensityResR1.txt", requirement_ph)
requirement_r1_high_nsb = load("IntensityResR1HighNSB.txt", requirement_ph)

def calculate_poisson_limit(pe):
    return np.sqrt(pe) / pe

# Resolution Calculator

The Intensity and Charge resolution of a camera can be estimated from the independant variance and bias components:

$$I = Q \times \epsilon_{PDE}$$
$$R_{TOTAL} = R_{NSB} + R_{DCR}$$
$$\epsilon_{ENF} = 1 + \epsilon_{OCT}$$

$$\sigma^2_{noise} = L \times (R_{TOTAL} \times \epsilon_{ENF} + \sigma_{WF}^2 )$$
$$\sigma^2_{ENF} = \epsilon_{ENF} \times Q$$
$$\sigma^2_{miscal} = \epsilon^2_{miscal} \times Q^2$$ 

$$\frac{\sigma_Q}{Q} = \frac{1}{Q}\sqrt{\sigma^2_{noise} + \sigma^2_{ENF} + \sigma^2_{miscal}}$$

In [4]:
class ResolutionCalculator:
    def __init__(self, optimal_overvoltage=4.5, gain_at_optimal=4, opct_at_optimal=None):
        self.eff = CameraEfficiency.from_sstcam()
        self.sipm = SiPMOvervoltage.lvr3_6mm_50um_uncoated()
        
        self.pde_ref_wavelength = u.Quantity(450, u.nm)
        self.optimal_overvolage = optimal_overvoltage
        self.noise_stddev_mv = 1.5
        self.window_width = 10
        
        self.sipm.scale_gain(self.optimal_overvolage, gain_at_optimal)
        if opct_at_optimal is not None:
            self.sipm.scale_opct(self.optimal_overvolage, opct_at_optimal)
        self.sipm.overvoltage = self.optimal_overvolage
        self.gain = self.sipm.gain
        self.opct = self.sipm.opct
        self.nsb_flux = 0.24
        self.dark_count_rate = 4
        self.pde_at_ref = self.sipm.pde
        self.miscal = 0.1
        self.saturation_limit_mv = 8000
        
        self.ph = np.geomspace(4, 4000, 100)
        
    @property
    def pe(self):
        return self.ph * self.pde_cherenkov
    
    @property
    def mv(self):
        return self.pe * self.mv_per_pe
        
    @property
    def gain(self):
        return self._gain
    
    @gain.setter
    def gain(self, value):
        self._gain = value
    
    @property
    def opct(self):
        return self._opct
    
    @opct.setter
    def opct(self, value):
        self._opct = value
    
    @property
    def pde_at_ref(self):
        return self._pde_at_ref

    @property
    def pde_cherenkov(self):
        return self._pde_cherenkov
    
    @pde_at_ref.setter
    def pde_at_ref(self, value):
        self._pde_at_ref = value
        self.eff.scale_pde(self.pde_ref_wavelength, value)
        self._pde_cherenkov = self.eff.camera_cherenkov_pde  # Update Cherenkov PDE
        self.nsb_flux = self._nsb_flux  # Update NSB

    @property
    def nsb_rate(self):
        return self._nsb_rate.to("MHz")
        
    @property
    def nsb_flux(self):
        return self._nsb_flux
        
    @nsb_flux.setter
    def nsb_flux(self, value):
        self._nsb_flux = value
        self._nsb_rate = self.eff.get_scaled_nsb_rate(u.Quantity(value, NSB_FLUX_UNIT))
        
    @property
    def dark_count_rate(self):
        return self._dark_count_rate
    
    @dark_count_rate.setter
    def dark_count_rate(self, value):
        self._dark_count_rate = value
        
    @property
    def miscal(self):
        return self._miscal

    @miscal.setter
    def miscal(self, value):
        self._miscal = value
        
    @property
    def mv_per_pe(self):
        fc_per_pe = 1/(1 - self.opct)
        return self.gain * fc_per_pe
    
    @property
    def mv_per_ph(self):
        return self.mv_per_pe * self.pde_cherenkov
    
    @property
    def noise_stddev_pe(self):
        return self.noise_stddev_mv / self.mv_per_pe
    
    @property
    def saturation_limit_mv(self):
        return self._saturation_limit_mv
    
    @saturation_limit_mv.setter
    def saturation_limit_mv(self, value):
        self._saturation_limit_mv = value

    @property
    def saturation_limit_pe(self):
        return self.saturation_limit_mv / self.mv_per_pe
    
    @property
    def saturation_limit_ph(self):
        return self.saturation_limit_pe / self.pde_cherenkov
    
    @property
    def enf(self):
        return 1 + self.opct
    
    @property
    def variance_noise(self):
        total_nsb = self.nsb_rate.to_value("MHz") + self.dark_count_rate
        return self.window_width * (total_nsb * 1e-3 * self.enf + self.noise_stddev_pe**2)
    
    @property
    def variance_enf(self):
        return self.enf * self.pe
    
    @property
    def variance_miscal(self):
        return (self.miscal * self.pe)**2
    
    @property
    def fractional_resolution(self):
        return np.sqrt(self.variance_noise + self.variance_enf + self.variance_miscal) / self.pe

In [15]:
calculator = ResolutionCalculator()

fig = plt.figure(figsize=(10, 5))
ax_intensity = fig.add_subplot(2, 1, 1)
ax_charge = fig.add_subplot(2, 1, 2)

l_intensity = ax_intensity.plot(calculator.ph, calculator.fractional_resolution)
l_charge = ax_charge.plot(calculator.pe, calculator.fractional_resolution)

# Poisson limit curves
poisson_limit = calculate_poisson_limit(calculator.pe)
l_intensity_poisson = ax_intensity.plot(calculator.ph, poisson_limit, ls='-', color='black', alpha=0.2)
l_charge_poisson = ax_charge.plot(calculator.pe, poisson_limit, ls='-', color='black', alpha=0.2)

# Requirement curves
ax_intensity.plot(requirement_ph, requirement_nominal_nsb, ls=':', color='black', alpha=0.2)
ax_intensity.plot(requirement_ph, requirement_high_nsb, ls='--', color='black', alpha=0.2)
ax_charge.plot(requirement_pe, requirement_nominal_nsb, ls=':', color='black', alpha=0.2)
ax_charge.plot(requirement_pe, requirement_high_nsb, ls='--', color='black', alpha=0.2)

# Saturation limit
l_intensity_saturation = ax_intensity.axvline(calculator.saturation_limit_ph, ls='--')
l_charge_saturation = ax_charge.axvline(calculator.saturation_limit_pe, ls='--')

# Annotations
t_nsb = fig.text(0.1, 0.93, f"NSB Rate = {calculator.nsb_rate:.2f}", transform=fig.transFigure)
t_pde = fig.text(0.3, 0.93, f"Cherenkov PDE = {calculator.pde_cherenkov:.2f}", transform=fig.transFigure)
t_mvperpe = fig.text(0.5, 0.93, f"mV/p.e. = {calculator.mv_per_pe:.2f}", transform=fig.transFigure)
t_mvperph = fig.text(0.65, 0.93, f"mV/photon = {calculator.mv_per_ph:.2f}", transform=fig.transFigure)


ax_intensity.set_xscale("log")
ax_intensity.set_yscale("log")
ax_charge.set_xscale("log")
ax_charge.set_yscale("log")

ax_intensity.set_xlabel("Intensity (photons)")
ax_intensity.set_ylabel("Fractional Intensity Resolution")
ax_charge.set_xlabel("Charge (p.e.)")
ax_charge.set_ylabel("Fractional Charge Resoltion")

fig.tight_layout()

def update(gain, opct, pde_at_ref, nsb_flux, dark_count_rate, noise_stddev_mv, miscal, saturation_limit):
    calculator.gain = gain
    calculator.opct = opct
    calculator.pde_at_ref = pde_at_ref
    calculator.nsb_flux = nsb_flux
    calculator.dark_count_rate = dark_count_rate
    calculator.noise_stddev_mv = noise_stddev_mv
    calculator.miscal = miscal
    calculator.saturation_limit_mv = saturation_limit
        
    # Update resolution curves
    l_intensity[0].set_xdata(calculator.ph)
    l_intensity[0].set_ydata(calculator.fractional_resolution)
    l_charge[0].set_xdata(calculator.pe)
    l_charge[0].set_ydata(calculator.fractional_resolution)
    
    # Update poisson limit curves
    poisson_limit = calculate_poisson_limit(calculator.pe)
    l_intensity_poisson[0].set_xdata(calculator.ph)
    l_intensity_poisson[0].set_ydata(poisson_limit)
    l_charge_poisson[0].set_xdata(calculator.pe)
    l_charge_poisson[0].set_ydata(poisson_limit)
    
    # Update saturation limit
    l_intensity_saturation.set_xdata(calculator.saturation_limit_ph)
    l_charge_saturation.set_xdata(calculator.saturation_limit_pe)
    
    # Update annotations
    t_nsb.set_text(f"NSB Rate = {calculator.nsb_rate:.2f}")
    t_pde.set_text(f"Cherenkov PDE = {calculator.pde_cherenkov:.2f}")
    t_mvperpe.set_text(f"mV/p.e. = {calculator.mv_per_pe:.2f}")
    t_mvperph.set_text(f"mV/photon = {calculator.mv_per_ph:.2f}")
    
slider_gain = widgets.FloatSlider(
    value=calculator.gain,
    min=0,
    max=6,
    step=0.01,
    description=f'Gain (mV):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_opct = widgets.FloatSlider(
    value=calculator.opct,
    min=0,
    max=0.99,
    step=0.01,
    description=f'Total OCT Rate:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_pde = widgets.FloatSlider(
    value=calculator.pde_at_ref,
    min=0.01,
    max=1,
    step=0.01,
    description=f'PDE @ {calculator.pde_ref_wavelength}:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_nsb_flux = widgets.FloatSlider(
    value=calculator.nsb_flux,
    min=0,
    max=4.3,
    step=0.01,
    description='NSB Flux:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_dark_count = widgets.FloatSlider(
    value=calculator.dark_count_rate,
    min=0,
    max=30,
    step=0.01,
    description='Dark Count Rate (MHz):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_noise_stddev = widgets.FloatSlider(
    value=calculator.noise_stddev_mv,
    min=0,
    max=5,
    step=0.01,
    description='WF Noise Stddev (mV):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_miscal = widgets.FloatSlider(
    value=calculator.miscal,
    min=0,
    max=1,
    step=0.01,
    description='Miscalibration:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_saturation = widgets.FloatSlider(
    value=calculator.saturation_limit_mv,
    min=2000,
    max=10000,
    step=100,
    description='Saturation Recovery Limit (mV):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
interactive(
    update, 
    gain=slider_gain, 
    opct=slider_opct, 
    pde_at_ref=slider_pde, 
    nsb_flux=slider_nsb_flux,
    dark_count_rate=slider_dark_count,
    noise_stddev_mv=slider_noise_stddev,
    miscal=slider_miscal,
    saturation_limit=slider_saturation,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=4.0, description='Gain (mV):', layout=Layout(width='50%'), max=6.0, st…

In [19]:
class ResolutionCalculatorSiPMDatasheet(ResolutionCalculator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._overvoltage_drop = 0
        self.intended_overvoltage = self.optimal_overvolage
        self.calibration_uncertainty = 0.07

    @property
    def overvoltage(self):
        return self.sipm.overvoltage
    
    @overvoltage.setter
    def overvoltage(self, value):
        self.sipm.overvoltage = value
        self.gain = self.sipm.gain
        self.opct = self.sipm.opct
        self.pde_at_ref = self.sipm.pde
        
    @property
    def intended_overvoltage(self):
        return self._intended_overvoltage
    
    @intended_overvoltage.setter
    def intended_overvoltage(self, value):
        self._intended_overvoltage = value
        
        # Obtain mV_per_ph at intended overvoltage
        self.overvoltage = value
        self._intended_mv_per_ph = self.mv_per_ph
        
        self.overvoltage = value - self.overvoltage_drop
        
    @property
    def overvoltage_drop(self):
        return self._overvoltage_drop
    
    @overvoltage_drop.setter
    def overvoltage_drop(self, value):
        self._overvoltage_drop = value
        self.overvoltage = self.intended_overvoltage - value
        
    @property
    def mv_per_ph_drop(self):
        return (self._intended_mv_per_ph - self.mv_per_ph) / self._intended_mv_per_ph
    
    @property
    def calibration_uncertainty(self):
        return self._calibration_uncertainty
    
    @calibration_uncertainty.setter
    def calibration_uncertainty(self, value):
        self._calibration_uncertainty = value
    
    @property
    def miscal(self):
        return (1 + self.calibration_uncertainty) * (1 + self.mv_per_ph_drop) - 1

    @miscal.setter
    def miscal(self, value):
        pass

In [21]:
calculator = ResolutionCalculatorSiPMDatasheet(
    optimal_overvoltage=6,
    gain_at_optimal=3.2,
    opct_at_optimal=0.05
)

fig = plt.figure(figsize=(9, 5))
ax_sipm_gain = fig.add_subplot(2, 2, 1)
ax_sipm_percent = ax_sipm_gain.twinx()
ax_intensity = fig.add_subplot(2, 2, 2)
ax_charge = fig.add_subplot(2, 2, 4)

l_intensity = ax_intensity.plot(calculator.ph, calculator.fractional_resolution)
l_charge = ax_charge.plot(calculator.pe, calculator.fractional_resolution)

# SiPM Curves
l_gain = ax_sipm_gain.plot(calculator.sipm._overvoltage_array, calculator.sipm._gain_array, color='green', label="Gain")
l_opct = ax_sipm_percent.plot(calculator.sipm._overvoltage_array, calculator.sipm._opct_array, color='red', label="OCT")
l_pde = ax_sipm_percent.plot(calculator.sipm._overvoltage_array, calculator.sipm._pde_array, color='blue', label="PDE")
l_intended = ax_sipm_gain.axvline(calculator.intended_overvoltage, color='black', ls=':', label="Expected OV")
l_actual = ax_sipm_gain.axvline(calculator.sipm.overvoltage, color='black', label="Actual OV")

# Poisson limit curves
poisson_limit = calculate_poisson_limit(calculator.pe)
l_intensity_poisson = ax_intensity.plot(calculator.ph, poisson_limit, ls='-', color='black', alpha=0.2)
l_charge_poisson = ax_charge.plot(calculator.pe, poisson_limit, ls='-', color='black', alpha=0.2)

# Requirement curves
l_ir_req_nominal = ax_intensity.plot(requirement_ph, requirement_nominal_nsb, ls=':', color='black', alpha=0.2)
l_ir_req_high = ax_intensity.plot(requirement_ph, requirement_high_nsb, ls='--', color='black', alpha=0.2)
l_cr_req_nominal = ax_charge.plot(requirement_pe, requirement_nominal_nsb, ls=':', color='black', alpha=0.2)
l_cr_req_high = ax_charge.plot(requirement_pe, requirement_high_nsb, ls='--', color='black', alpha=0.2)

# Saturation limit
l_intensity_saturation = ax_intensity.axvline(calculator.saturation_limit_ph, ls='--')
l_charge_saturation = ax_charge.axvline(calculator.saturation_limit_pe, ls='--')

# Annotations
t_mvperfc = fig.text(0.1, 0.40, f"mV/f.c. (Gain) = {calculator.gain:.2f} V", transform=fig.transFigure)
t_opct = fig.text(0.1, 0.37, f"OPCT = {calculator.opct:.2f}", transform=fig.transFigure)
t_pdeat = fig.text(0.1, 0.34, f"PDE @ {calculator.pde_ref_wavelength} = {calculator.pde_at_ref:.2f}", transform=fig.transFigure)
t_expected_ov = fig.text(0.1, 0.31, f"Expected Overvoltage = {calculator.intended_overvoltage:.2f} V", transform=fig.transFigure)
t_actual_ov = fig.text(0.1, 0.28, f"Actual Overvoltage = {calculator.sipm.overvoltage:.2f} V", transform=fig.transFigure)
t_expected_mvperph = fig.text(0.1, 0.25, f"Expected mV/photon = {calculator._intended_mv_per_ph:.2f}", transform=fig.transFigure)
t_actual_mvperph = fig.text(0.1, 0.22, f"Actual mV/photon = {calculator.mv_per_ph:.2f}", transform=fig.transFigure)
t_nsb = fig.text(0.1, 0.15, f"NSB Rate = {calculator.nsb_rate:.2f}", transform=fig.transFigure)
t_pde = fig.text(0.1, 0.12, f"Cherenkov PDE = {calculator.pde_cherenkov:.2f}", transform=fig.transFigure)
t_mvperpe = fig.text(0.1, 0.09, f"mV/p.e. = {calculator.mv_per_pe:.2f}", transform=fig.transFigure)
t_miscal = fig.text(0.1, 0.06, f"Total Miscalibration = {calculator.miscal:.3f}", transform=fig.transFigure)

ax_intensity.set_xscale("log")
ax_intensity.set_yscale("log")
ax_charge.set_xscale("log")
ax_charge.set_yscale("log")

ax_intensity.set_xlabel("Intensity (photons)")
ax_intensity.set_ylabel("Fractional Intensity Resolution")
ax_charge.set_xlabel("Charge (p.e.)")
ax_charge.set_ylabel("Fractional Charge Resoltion")

ax_sipm_gain.set_xlabel("Overvoltage (V)")
ax_sipm_gain.set_ylabel("Gain (mV/f.c.)")
ax_sipm_percent.set_ylabel("OCT & PDE")

lns = l_gain + l_opct + l_pde + [l_intended] + [l_actual]
labs = [l.get_label() for l in lns]
ax_sipm_gain.legend(lns, labs, loc='best', fontsize=5)

fig.tight_layout()


def update(intended_overvoltage, overvoltage_drop, nsb_flux, calibration_uncertainty, online=False):
    calculator.intended_overvoltage = intended_overvoltage
    calculator.overvoltage_drop = overvoltage_drop
    calculator.nsb_flux = nsb_flux
    calculator.calibration_uncertainty = calibration_uncertainty
        
    # Update resolution curves
    l_intensity[0].set_xdata(calculator.ph)
    l_intensity[0].set_ydata(calculator.fractional_resolution)
    l_charge[0].set_xdata(calculator.pe)
    l_charge[0].set_ydata(calculator.fractional_resolution)
    
    # Update SiPM curves
    l_intended.set_xdata(calculator.intended_overvoltage)
    l_actual.set_xdata(calculator.sipm.overvoltage)
    
    # Update poisson limit curves
    poisson_limit = calculate_poisson_limit(calculator.pe)
    l_intensity_poisson[0].set_xdata(calculator.ph)
    l_intensity_poisson[0].set_ydata(poisson_limit)
    l_charge_poisson[0].set_xdata(calculator.pe)
    l_charge_poisson[0].set_ydata(poisson_limit)
    
    # Update saturation limit
    l_intensity_saturation.set_xdata(calculator.saturation_limit_ph)
    l_charge_saturation.set_xdata(calculator.saturation_limit_pe)
    
    # Update annotations
    t_mvperfc.set_text(f"mV/f.c. (Gain) = {calculator.gain:.2f} V")
    t_opct.set_text(f"OPCT = {calculator.opct:.2f}")
    t_pdeat.set_text(f"PDE @ {calculator.pde_ref_wavelength} = {calculator.pde_at_ref:.2f}")
    t_expected_ov.set_text(f"Expected Overvoltage = {calculator.intended_overvoltage:.2f} V")
    t_actual_ov.set_text(f"Actual Overvoltage = {calculator.sipm.overvoltage:.2f} V")
    t_expected_mvperph.set_text(f"Expected mV/photon = {calculator._intended_mv_per_ph:.2f}")
    t_actual_mvperph.set_text(f"Actual mV/photon = {calculator.mv_per_ph:.2f}")
    t_nsb.set_text(f"NSB Rate = {calculator.nsb_rate:.2f}")
    t_pde.set_text(f"Cherenkov PDE = {calculator.pde_cherenkov:.2f}")
    t_mvperpe.set_text(f"mV/p.e. = {calculator.mv_per_pe:.2f}")   
    t_miscal.set_text(f"Total Miscalibration = {calculator.miscal:.3f}")

    if online:
        l_ir_req_nominal[0].set_ydata(requirement_r1_nominal_nsb)
        l_ir_req_high[0].set_ydata(requirement_r1_high_nsb)
        l_cr_req_nominal[0].set_ydata(requirement_r1_nominal_nsb)
        l_cr_req_high[0].set_ydata(requirement_r1_high_nsb)
    else:
        l_ir_req_nominal[0].set_ydata(requirement_nominal_nsb)
        l_ir_req_high[0].set_ydata(requirement_high_nsb)
        l_cr_req_nominal[0].set_ydata(requirement_nominal_nsb)
        l_cr_req_high[0].set_ydata(requirement_high_nsb)
    
slider_intended = widgets.FloatSlider(
    value=calculator.intended_overvoltage,
    min=calculator.sipm._overvoltage_array.min(),
    max=calculator.sipm._overvoltage_array.max(),
    step=0.01,
    description=f'Intended Overvoltage (V):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_drop = widgets.FloatSlider(
    value=calculator.overvoltage_drop,
    min=0,
    max=3,
    step=0.01,
    description=f'Overvoltage Drop (V):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_nsb_flux = widgets.FloatSlider(
    value=calculator.nsb_flux,
    min=0,
    max=4.3,
    step=0.01,
    description='NSB Flux:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
slider_calib = widgets.FloatSlider(
    value=calculator.calibration_uncertainty,
    min=0,
    max=0.1,
    step=0.001,
    description='Calibration Uncertainty:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
checkbox_online = widgets.Checkbox(
    value=False,
    description='Online (R1) Requirements',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
interactive(
    update, 
    intended_overvoltage=slider_intended, 
    overvoltage_drop=slider_drop, 
    nsb_flux=slider_nsb_flux,
    calibration_uncertainty=slider_calib,
    online=checkbox_online,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=6.0, description='Intended Overvoltage (V):', layout=Layout(width='50%…